# BGP Anomaly Detection using SVM

This notebook explores how to use Support Vector Machines (SVM) for detecting BGP routing anomalies using the feature-rich datasets extracted by the BGP Feature Extractor tool.

## What is BGP?

Border Gateway Protocol (BGP) is the routing protocol that enables the global internet to function by allowing autonomous systems (ASes) to exchange network reachability information. BGP anomalies can include route hijacks, leaks, configuration errors, or infrastructure failures that disrupt normal internet routing.

## What are we doing in this notebook?

1. Load and explore labeled BGP features from a dataset
2. Preprocess the data for machine learning
3. Train an SVM model to detect anomalies
4. Evaluate the model's performance using ROC curves, precision-recall curves, etc.
5. Explain model predictions using SHAP and LIME
6. Visualize the results

In [ ]:
!pip install shap
!pip install lime

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, 
    precision_recall_fscore_support, roc_curve, auc, 
    precision_recall_curve, average_precision_score
)
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import joblib
import warnings

# Import explainability libraries if installed
try:
    import shap
    import lime
    import lime.lime_tabular
    EXPLAINABILITY_AVAILABLE = True
except ImportError:
    EXPLAINABILITY_AVAILABLE = False
    print("Warning: SHAP and/or LIME libraries not installed. Explainability sections will be skipped.")
    print("Run 'pip install shap lime' to install them.")

# Suppress warnings
warnings.filterwarnings('ignore')

# Set plot style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)

## 1. Loading the Dataset

We'll load a dataset that contains BGP features extracted from an anomaly event. For this example, we'll use a dataset from the Code Red worm incident, which caused significant routing disruption.

In [ ]:
# Load the dataset - update the path to the dataset file you want to use
dataset_path = "datasets/ratios/dataset_nimda_513_5_rrc04.csv"
df = pd.read_csv(dataset_path)

# Check the first few rows
print(f"Dataset shape: {df.shape}")
df.head()

## 2. Data Exploration and Preprocessing

In [ ]:
# Check basic statistics
df.describe()

In [ ]:
# Check for missing values
print("Missing values in each column:")
df.isnull().sum()

In [ ]:
# Fill missing values with 0 (if any)
df = df.fillna(0)

In [ ]:
# Check class distribution
print("Class distribution:")
print(df['class'].value_counts())

# Plot class distribution
plt.figure(figsize=(8, 5))
sns.countplot(x='class', data=df)
plt.title('Class Distribution (0 = Normal, 1 = Anomaly)')
plt.show()

In [ ]:
# Drop non-feature columns
columns_to_drop = ['timestamp', 'timestamp2', 'Unnamed: 0']
for col in columns_to_drop:
    if col in df.columns:
        df = df.drop(col, axis=1)

# Separate features and target
X = df.drop('class', axis=1)
y = df['class']

# Display feature names
print(f"Number of features: {X.shape[1]}")
print("Feature names:")
X.columns.tolist()

## 3. Feature Analysis and Selection

In [ ]:
# Analyze feature importance for anomaly detection
# For each feature, calculate the mean difference between normal and anomaly periods
anomaly = df[df['class'] == 1]
normal = df[df['class'] == 0]

# Calculate mean differences
feature_diffs = {}
for col in X.columns:
    if col != 'class':
        normal_mean = normal[col].mean()
        anomaly_mean = anomaly[col].mean() if not anomaly.empty else 0
        
        # Prevent division by zero
        if normal_mean == 0:
            if anomaly_mean == 0:
                diff = 0
            else:
                diff = float('inf')
        else:
            diff = (anomaly_mean - normal_mean) / normal_mean
        
        feature_diffs[col] = diff

# Convert to DataFrame for better visualization
feature_diff_df = pd.DataFrame(list(feature_diffs.items()), columns=['Feature', 'Relative Change'])
feature_diff_df = feature_diff_df.sort_values('Relative Change', key=abs, ascending=False)

# Plot top features by absolute difference
plt.figure(figsize=(14, 10))
sns.barplot(x='Relative Change', y='Feature', data=feature_diff_df.head(15))
plt.title('Top 15 Features by Absolute Relative Change Between Normal and Anomaly Periods')
plt.tight_layout()
plt.show()

In [ ]:
# Select top features based on difference analysis
top_features = feature_diff_df.head(15)['Feature'].tolist()
print("Selected top features:")
print(top_features)

# Extract selected features from dataset
X_selected = X[top_features]

## 4. Train-Test Split

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

## 5. SVM Model Training and Hyperparameter Tuning

In [ ]:
# Create a pipeline with preprocessing and SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True))
])

# Define the hyperparameter search space
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto', 0.1, 0.01],
    'svm__kernel': ['rbf', 'linear', 'poly']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', verbose=1)
grid_search.fit(X_train, y_train)

# Print best parameters
print("Best parameters found:")
print(grid_search.best_params_)
print(f"Best CV score: {grid_search.best_score_:.4f}")

In [ ]:
# Get the best model
best_model = grid_search.best_estimator_

# Evaluate on test set
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

## 6. Model Evaluation with Advanced Metrics

In [ ]:
# Calculate and print evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Normal', 'Anomaly'],
            yticklabels=['Normal', 'Anomaly'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# Calculate and plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
# Calculate and plot Precision-Recall curve
precision_values, recall_values, _ = precision_recall_curve(y_test, y_proba)
avg_precision = average_precision_score(y_test, y_proba)

plt.figure(figsize=(10, 8))
plt.plot(recall_values, precision_values, color='blue', lw=2, 
         label=f'Precision-Recall curve (AP = {avg_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.grid(True)
plt.show()

## 7. Visualizing Decision Boundaries

Let's use PCA to reduce our features to 2 dimensions and visualize the SVM decision boundary.

In [ ]:
# Apply PCA to reduce to 2 dimensions for visualization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Explained variance
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {sum(pca.explained_variance_ratio_):.2f}")

# Create a DataFrame for easier plotting
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['class'] = y.values

# Plot PCA results
plt.figure(figsize=(12, 10))
sns.scatterplot(x='PC1', y='PC2', hue='class', data=pca_df, palette='Set1', s=100, alpha=0.7)
plt.title('PCA of BGP Features (2 Components)')
plt.legend(title='Class', labels=['Normal', 'Anomaly'])
plt.show()

In [ ]:
# Train an SVM on the PCA-transformed data
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
    X_pca, y, test_size=0.3, random_state=42, stratify=y
)

# Use best parameters from grid search but adapt to the 2D case
best_params = grid_search.best_params_
svm_2d = SVC(
    C=best_params['svm__C'],
    gamma=best_params['svm__gamma'] if best_params['svm__gamma'] not in ['scale', 'auto'] else 'scale',
    kernel=best_params['svm__kernel'],
    probability=True
)
svm_2d.fit(X_train_pca, y_train_pca)

# Plot decision boundary
def plot_decision_boundary(X, y, model, ax=None):
    h = 0.02  # Step size in the mesh
    
    # Create mesh grid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Get predictions for each point in the mesh
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot
    if ax is None:
        plt.figure(figsize=(12, 10))
        ax = plt.gca()
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', s=100, alpha=0.7, edgecolors='k')
    return ax

plt.figure(figsize=(14, 12))
ax = plot_decision_boundary(X_pca, y.values, svm_2d)
plt.title(f'SVM Decision Boundary (Kernel: {best_params["svm__kernel"]})')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

## 8. Anomaly Score Time Series

We can plot the prediction probabilities (anomaly scores) as a time series to see how the model's confidence varies over time.

In [ ]:
# Load full dataset again to get timestamps
full_df = pd.read_csv(dataset_path)
timestamps = full_df['timestamp']

# Predict probabilities on all data
X_all_selected = X_selected.copy()
all_proba = best_model.predict_proba(X_all_selected)[:, 1]

# Create time series dataframe
time_series_df = pd.DataFrame({
    'timestamp': timestamps,
    'anomaly_score': all_proba,
    'true_class': y
})

# Convert timestamp strings to datetime objects if not already
time_series_df['timestamp'] = pd.to_datetime(time_series_df['timestamp'])

# Plot anomaly score time series
plt.figure(figsize=(14, 8))
plt.plot(time_series_df['timestamp'], time_series_df['anomaly_score'], 'b-', label='Anomaly Score')

# Highlight actual anomaly periods
anomaly_periods = time_series_df[time_series_df['true_class'] == 1]
for idx, row in anomaly_periods.iterrows():
    plt.axvline(x=row['timestamp'], color='r', linestyle='--', alpha=0.3)

# Add horizontal line at threshold 0.5
plt.axhline(y=0.5, color='g', linestyle='-', label='Decision Threshold')

plt.title('BGP Anomaly Detection Time Series')
plt.xlabel('Time')
plt.ylabel('Anomaly Score (Probability)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 9. Feature Importance Analysis with Linear SVM

In [ ]:
# Create and train a linear SVM for feature importance analysis
linear_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='linear'))
])
linear_svm.fit(X_train, y_train)

# Extract feature importance scores (coefficients)
importances = abs(linear_svm.named_steps['svm'].coef_[0])
feature_importance = pd.DataFrame({'Feature': X_selected.columns, 'Importance': importances})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(14, 10))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Feature Importance (Linear SVM Coefficients)')
plt.tight_layout()
plt.show()

## 10. SHAP Values for Model Explainability

SHAP (SHapley Additive exPlanations) values allow us to understand how each feature contributes to individual predictions.

In [ ]:
if EXPLAINABILITY_AVAILABLE:
    # Create a simpler model for SHAP analysis (to speed up computation)
    shap_model = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='linear', probability=True))
    ])
    shap_model.fit(X_train, y_train)
    
    # Use a subset of training data as background for the explainer
    X_train_sample = X_train.sample(min(100, len(X_train)), random_state=42)
    X_train_transformed = shap_model.named_steps['scaler'].transform(X_train_sample)
    
    # Create a SHAP KernelExplainer
    def f(x):
        return shap_model.predict_proba(x)[:,1]
    
    explainer = shap.KernelExplainer(f, X_train_transformed)
    
    # Select a sample of test instances to explain
    X_test_sample = X_test.sample(min(10, len(X_test)), random_state=42)
    X_test_transformed = shap_model.named_steps['scaler'].transform(X_test_sample)
    
    # Compute SHAP values
    shap_values = explainer.shap_values(X_test_transformed)
    
    # Summary plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_test_sample, feature_names=X_selected.columns)
    
    # SHAP dependence plot for top feature
    top_feature_idx = np.argmax(np.abs(explainer.expected_value))
    top_feature = X_selected.columns[top_feature_idx]
    
    plt.figure(figsize=(12, 8))
    shap.dependence_plot(top_feature_idx, shap_values, X_test_transformed, 
                         feature_names=X_selected.columns)
else:
    print("SHAP analysis skipped because libraries are not installed.")

## 11. LIME for Local Interpretability

LIME (Local Interpretable Model-agnostic Explanations) helps understand what features are most important for a specific prediction.

In [ ]:
if EXPLAINABILITY_AVAILABLE:
    # Create a LIME explainer
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        X_train.values,
        feature_names=X_selected.columns.tolist(),
        class_names=['Normal', 'Anomaly'],
        verbose=True,
        mode='classification'
    )
    
    # Find an anomaly instance to explain
    anomaly_indices = np.where(y_test == 1)[0]
    if len(anomaly_indices) > 0:
        # Pick an anomaly in the test set
        anomaly_idx = anomaly_indices[0]
        anomaly_instance = X_test.iloc[anomaly_idx].values
        
        # Generate LIME explanation
        lime_exp = lime_explainer.explain_instance(
            anomaly_instance, 
            best_model.predict_proba,
            num_features=10
        )
        
        # Plot the explanation
        plt.figure(figsize=(12, 6))
        lime_exp.as_pyplot_figure()
        plt.title("LIME Explanation for Anomaly Instance")
        plt.tight_layout()
        plt.show()
    else:
        print("No anomalies found in test set")
else:
    print("LIME analysis skipped because libraries are not installed.")

## 12. Model Persistence

Save the trained model for future use.

In [ ]:
# Save the model
joblib.dump(best_model, 'svm_nimda_5min_model.pkl')
print("Model saved to 'bgp_anomaly_svm_model.pkl'")

# Save the feature list
pd.DataFrame(top_features, columns=['Feature']).to_csv('nimda_5_bgp_anomaly_selected_features.csv', index=False)
print("Selected features saved to 'bgp_anomaly_selected_features.csv'")

## 13. Loading the Model for Future Predictions

Here's how you would load the model and make predictions on new data.

In [ ]:
# Example of how to load and use the model
loaded_model = joblib.load('svm_nimda_5min_model.pkl')
selected_features = pd.read_csv('nimda_5_bgp_anomaly_selected_features.csv')['Feature'].tolist()

# Example with new data (replace with actual new data)
# new_data = pd.read_csv('new_bgp_data.csv')
# X_new = new_data[selected_features]
# predictions = loaded_model.predict(X_new)
# anomaly_scores = loaded_model.predict_proba(X_new)[:, 1]

print("Model and feature list loaded successfully.")

## 14. Conclusion

In this notebook, we've built an SVM-based anomaly detection system for BGP routing data with comprehensive evaluation and explanations. The model can effectively distinguish between normal BGP traffic and anomalous events based on the features extracted by the BGP Feature Extractor tool.

Key observations:
1. Feature selection is crucial - we identified the most discriminative features
2. SVM with appropriate hyperparameters can achieve high accuracy in detecting anomalies
3. ROC curves and precision-recall curves provide deeper insights into model performance
4. SHAP and LIME explain what features contribute most to anomaly predictions
5. Visualizing the time series of anomaly scores helps understand model behavior over time

This model could be deployed as part of a network monitoring system to provide early warning of potential BGP anomalies, helping to secure internet routing infrastructure.